In [43]:
a = 1

1

In [ ]:
##############################################################################
# Created by: Wesley DA SILVA COELHO
##############################################################################

##############################################################################
#                   INCLUDE PACKAGES
##############################################################################
using NBInclude #Package to include personal files",
using JuMP #Mathematical programming package : allows us to define the MILP",
using LightGraphs #Package to construct simple graphs",
using MetaGraphs #Package to construct complex graphs",
using GraphPlot

# path to some folders
input_folder = "../instances/heuristic/tiny/"
include_folder = "../include/"
result_folder = "../instances/heuristic/tiny/results/" 
@nbinclude(joinpath(include_folder,"NSDP_instance_generator.ipynb"))# code creator for 5GNSD instances
@nbinclude(joinpath(include_folder,"NSDP_structures.ipynb")) # our data structures 
@nbinclude(joinpath(include_folder,"NSDP_instance_reader.ipynb"))# file responsible for parseing new NSDP instances
@nbinclude(joinpath(include_folder,"NSDP_heuristic.ipynb")) # impementation of our model 



my_VIs = ["none"]
my_cuts = ["none"]
my_solvers = ["CPLEX"]
my_OFs = ["minNFS"]
boosts = [true]
my_parameters = Parameters()

my_instance_sizes = get_my_instance_sizes()
latency = ["low","high"]
isolation = ["strong","weak"]
capacities = ["tight","moderate"]
instances_type = 0
number_of_instances = 0
exp_re = 30
open_file_stat(exp_re)
      
    open(joinpath(result_folder,"load_on_arcs.csv"), "w") do iooo
    open(joinpath(result_folder,"final_statistics.csv"), "w") do io
    open(joinpath(result_folder,"load_on_nodes.csv"), "w") do ioo    
        write(io,"instance;approach;class;graph_density;link_capacity_ratio;prob_sharing_NFS;")
        write(ioo,"instance;approach;node;node_type;class;graph_density;link_capacity_ratio;charge_node;prob_sharing_NFS\n")
        write(iooo,"instance;approach;arc;arc_type;class;graph_density;link_capacity_ratio;charge_arc;prob_sharing_NFS\n")
                write(io,"Objective_value;")   
                write(io,"number_NFSs;")
                write(io,"dis_NFS;")
                write(io,"cent_NFS;")


                write(io,"number_nodes;")
                write(io,"number_active_node;")
                write(io,"proportion_active_node;")

                write(io,"total_nodes_capacity;")
                write(io,"total_load_node;")
                write(io,"av_node_load);")
                write(io,"most_loaded_node;")


                write(io,"number_of_arcs;")
                write(io,"number_active_links;")
                write(io,"propotyion_number_active_links;")

                write(io,"total_arc_capacity;")
                write(io,"total_arc_load;")
                write(io,"av_link_load;")
                write(io,"most_loaded_arc;")
                write(io,"avr_lat\n")
        end
        end
        end

for instance_size in 1:1, lat in latency, iso in isolation, c in capacities

    size = my_instance_sizes[instance_size]
    instances_type = instances_type + 1
    n = size["nodes"]
    d = size["densitiy"]
   if  iso == "strong"
        nfsh  = 0.1
        nosh = 0.1
    else
        nfsh  = 0.75
        nosh = 0.75
    end
        
    s  = size["slices"]
    k = size["commodities"]
    dp = size["DP_NFSs"]
    cp = size["CP_NFSs"]

    tries = 0
    i = 0
        
    while i <30
        i = i+1

        #creating instances
        number_of_instances = number_of_instances+1
        instance = get_Instance(input_folder,number_of_instances)  
        warming_up =  "FEASIBLE"#warm_up(my_instance,"warm_tiny")
        

        if warming_up != "FEASIBLE" && warming_up != "OPTIMAL" && tries<=2000
            tries = tries+1
            i = i-1
            number_of_instances=number_of_instances-1
        elseif warming_up != "FEASIBLE" && warming_up != "OPTIMAL" && tries>2000
            i = 5000 
            number_of_instances=number_of_instances-1    
        elseif warming_up == "FEASIBLE" || warming_up == "OPTIMAL"
            tries = 0
            #solving instances
            for of in my_OFs, b in boosts, sol in my_solvers, vi in my_VIs,cut in my_cuts
                my_parameters = Parameters()
                my_parameters.class = "$(size["class"])_$(c)_$(lat)_$(iso)"            
                my_parameters.instance_name = "instance_$(number_of_instances)_of_$(of)_boost_$(b)_solver_$(sol)_vi_$(vi)_cut_$(cut)_$(my_parameters.class)"              
                my_parameters.valid_inequalities = vi
                my_parameters.boost = b
                #my_parameters.instance_class = ""            
                my_parameters.solver = sol
                my_parameters.instance_type = "instance_$(number_of_instances)_type_$(instances_type)_of_$(of)"
                my_parameters.instance_number = number_of_instances
                my_parameters.warming_up = false
                my_parameters.number_of_phy_nodes = n
                my_parameters.graph_density = d
                my_parameters.link_capacity_ratio = c
                my_parameters.node_capacity_ratio =c 
                my_parameters.number_of_slice_requests =s
                my_parameters.number_of_commodities_per_NS = k
                my_parameters.number_of_CP_NFSs = cp 
                my_parameters.number_of_DP_NFSs = dp
                my_parameters.prob_sharing_NFS = nfsh
                my_parameters.prob_sharing_node = nosh
                my_parameters.objf = of                
                my_parameters.cut_class = cut
                my_parameters.cut_time = 0.0
                my_parameters.number_of_cuts = 0
                my_parameters.number_of_lazy_cuts= 0
                my_parameters.lazy_time = 0.0            


        #creating model

                #recreating new model cause jump starts from de LP already solved
                my_parameters.relaxation=false
                model = create_NSDP_complete_model(instance,my_parameters)                
                # calling solver 
                solve_NSDP_model(model,my_parameters,instance,"")
               # get_MILP_LOG(result_folder,my_parameters,instance)
                export_stat_instance(my_parameters,exp_re,"complete")
                get_solution_mod(model,instance,result_folder,lat,number_of_instances,my_parameters)
                model = nothing 

                my_instance = solve_NSDP_heuristic(input_folder,number_of_instances, my_parameters.MILP_solution)
                export_stat_instance_heu(my_parameters,exp_re,my_instance,my_parameters.MILP_solution)
                get_solution_heu(my_instance,result_folder,lat,number_of_instances,my_parameters)
                
            end
        end
    end
end    


In [38]:
instance.

UndefRefError: UndefRefError: access to undefined reference

In [ ]:
@nbinclude(joinpath(include_folder,"NSDP_heuristic.ipynb")) # impementation of our model 
my_instance = solve_NSDP_heuristic(input_folder,number_of_instances, my_parameters.MILP_solution)


In [51]:
number_of_instances

6

In [ ]:
##############################################################################
# Created by: Wesley DA SILVA COELHO
##############################################################################

##############################################################################
#                   INCLUDE PACKAGES
##############################################################################
using NBInclude #Package to include personal files",
using JuMP #Mathematical programming package : allows us to define the MILP",
using LightGraphs #Package to construct simple graphs",
using MetaGraphs #Package to construct complex graphs",
using GraphPlot

# path to some folders
input_folder = "../instances/heuristic/small/"
include_folder = "../include/"
result_folder = "../instances/heuristic/small/results/" 
@nbinclude(joinpath(include_folder,"NSDP_instance_generator.ipynb"))# code creator for 5GNSD instances
@nbinclude(joinpath(include_folder,"NSDP_structures.ipynb")) # our data structures 
@nbinclude(joinpath(include_folder,"NSDP_instance_reader.ipynb"))# file responsible for parseing new NSDP instances
@nbinclude(joinpath(include_folder,"NSDP_heuristic.ipynb")) # impementation of our model 



my_VIs = ["none"]
my_cuts = ["none"]
my_solvers = ["CPLEX"]
my_OFs = ["minNFS"]
boosts = [true]
my_parameters = Parameters()

my_instance_sizes = get_my_instance_sizes()
latency = ["low","high"]
isolation = ["strong","weak"]
capacities = ["tight","moderate"]
instances_type = 0
number_of_instances = 0
exp_re = 30
open_file_stat(exp_re)
      
    open(joinpath(result_folder,"load_on_arcs.csv"), "w") do iooo
    open(joinpath(result_folder,"final_statistics.csv"), "w") do io
    open(joinpath(result_folder,"load_on_nodes.csv"), "w") do ioo    
        write(io,"instance;approach;class;graph_density;link_capacity_ratio;prob_sharing_NFS;")
        write(ioo,"instance;approach;node;node_type;class;graph_density;link_capacity_ratio;charge_node;prob_sharing_NFS\n")
        write(iooo,"instance;approach;arc;arc_type;class;graph_density;link_capacity_ratio;charge_arc;prob_sharing_NFS\n")
                write(io,"Objective_value;")   
                write(io,"number_NFSs;")
                write(io,"dis_NFS;")
                write(io,"cent_NFS;")


                write(io,"number_nodes;")
                write(io,"number_active_node;")
                write(io,"proportion_active_node;")

                write(io,"total_nodes_capacity;")
                write(io,"total_load_node;")
                write(io,"av_node_load);")
                write(io,"most_loaded_node;")


                write(io,"number_of_arcs;")
                write(io,"number_active_links;")
                write(io,"propotyion_number_active_links;")

                write(io,"total_arc_capacity;")
                write(io,"total_arc_load;")
                write(io,"av_link_load;")
                write(io,"most_loaded_arc;")
                write(io,"avr_lat\n")
        end
        end
        end

for instance_size in 2:2, lat in latency, iso in isolation, c in capacities

    size = my_instance_sizes[instance_size]
    instances_type = instances_type + 1
    n = size["nodes"]
    d = size["densitiy"]
   if  iso == "strong"
        nfsh  = 0.1
        nosh = 0.1
    else
        nfsh  = 0.75
        nosh = 0.75
    end
        
    s  = size["slices"]
    k = size["commodities"]
    dp = size["DP_NFSs"]
    cp = size["CP_NFSs"]

    tries = 0
    i = 0
        
    while i <30
        i = i+1

        #creating instances
        number_of_instances = number_of_instances+1
        instance = get_Instance(input_folder,number_of_instances)  
        warming_up =  "FEASIBLE"#warm_up(my_instance,"warm_tiny")
        

        if warming_up != "FEASIBLE" && warming_up != "OPTIMAL" && tries<=2000
            tries = tries+1
            i = i-1
            number_of_instances=number_of_instances-1
        elseif warming_up != "FEASIBLE" && warming_up != "OPTIMAL" && tries>2000
            i = 5000 
            number_of_instances=number_of_instances-1    
        elseif warming_up == "FEASIBLE" || warming_up == "OPTIMAL"
            tries = 0
            #solving instances
            for of in my_OFs, b in boosts, sol in my_solvers, vi in my_VIs,cut in my_cuts
                my_parameters = Parameters()
                my_parameters.class = "$(size["class"])_$(c)_$(lat)_$(iso)"            
                my_parameters.instance_name = "instance_$(number_of_instances)_of_$(of)_boost_$(b)_solver_$(sol)_vi_$(vi)_cut_$(cut)_$(my_parameters.class)"              
                my_parameters.valid_inequalities = vi
                my_parameters.boost = b
                #my_parameters.instance_class = ""            
                my_parameters.solver = sol
                my_parameters.instance_type = "instance_$(number_of_instances)_type_$(instances_type)_of_$(of)"
                my_parameters.instance_number = number_of_instances
                my_parameters.warming_up = false
                my_parameters.number_of_phy_nodes = n
                my_parameters.graph_density = d
                my_parameters.link_capacity_ratio = c
                my_parameters.node_capacity_ratio =c 
                my_parameters.number_of_slice_requests =s
                my_parameters.number_of_commodities_per_NS = k
                my_parameters.number_of_CP_NFSs = cp 
                my_parameters.number_of_DP_NFSs = dp
                my_parameters.prob_sharing_NFS = nfsh
                my_parameters.prob_sharing_node = nosh
                my_parameters.objf = of                
                my_parameters.cut_class = cut
                my_parameters.cut_time = 0.0
                my_parameters.number_of_cuts = 0
                my_parameters.number_of_lazy_cuts= 0
                my_parameters.lazy_time = 0.0            


        #creating model

                #recreating new model cause jump starts from de LP already solved
                my_parameters.relaxation=false
                model = create_NSDP_complete_model(instance,my_parameters)                
                # calling solver 
                solve_NSDP_model(model,my_parameters,instance,"")
               # get_MILP_LOG(result_folder,my_parameters,instance)
                export_stat_instance(my_parameters,exp_re,"complete")
                get_solution_mod(model,instance,result_folder,lat,number_of_instances,my_parameters)
                model = nothing 

                my_instance = solve_NSDP_heuristic(input_folder,number_of_instances)
                export_stat_instance_heu(my_parameters,exp_re,my_instance,my_parameters.MILP_solution)
                get_solution_heu(my_instance,result_folder,lat,number_of_instances,my_parameters)
                
            end
        end
    end
end    


Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Threads                                 12
CPXPARAM_MIP_Display                             4
CPXPARAM_MIP_Strategy_File                       2
CPXPARAM_MIP_Interval                            100000
CPXPARAM_TimeLimit                               10800
CPXPARAM_WorkMem                                 10240
Tried aggregator 3 times.
MIP Presolve eliminated 1047464 rows and 201235 columns.
MIP Presolve modified 155064 coefficients.
Aggregator did 108244 substitutions.
Reduced MIP has 131777 rows, 147977 columns, and 1320197 nonzeros.
Reduced MIP has 127977 binaries, 20000 generals, 0 SOSs, and 0 indicators.
Presolve time = 14.21 sec. (8010.07 ticks)
Elapsed time = 15.48 sec. (10001.65 ticks) for 18% of probing (2672 vars fixed)
Probing fixed 20624 vars, tightened 1000 bounds.
Probing time = 28.14 sec. (16332.32 ticks)
Tried aggregator 4 times.
MIP Presolve eliminated 9707 rows and 32894 columns.
MIP Presolve modified 19787 

In [19]:
number_of_instances

10

In [ ]:
@nbinclude(joinpath(include_folder,"NSDP_heuristic.ipynb")) # impementation of our model 
##############################################################################
# Created by: Wesley DA SILVA COELHO
##############################################################################

##############################################################################
#                   INCLUDE PACKAGES
##############################################################################
using NBInclude #Package to include personal files",
using JuMP #Mathematical programming package : allows us to define the MILP",
using LightGraphs #Package to construct simple graphs",
using MetaGraphs #Package to construct complex graphs",
using GraphPlot

# path to some folders
input_folder = "../instances/heuristic/small/"
include_folder = "../include/"
result_folder = "../instances/heuristic/small/results/" 
@nbinclude(joinpath(include_folder,"NSDP_instance_generator.ipynb"))# code creator for 5GNSD instances
@nbinclude(joinpath(include_folder,"NSDP_structures.ipynb")) # our data structures 
@nbinclude(joinpath(include_folder,"NSDP_instance_reader.ipynb"))# file responsible for parseing new NSDP instances
@nbinclude(joinpath(include_folder,"NSDP_heuristic.ipynb")) # impementation of our model 



my_VIs = ["none"]
my_cuts = ["none"]
my_solvers = ["CPLEX"]
my_OFs = ["minNFS"]
boosts = [true]
my_parameters = Parameters()

my_instance_sizes = get_my_instance_sizes()
latency = ["low","high"]
isolation = ["strong","weak"]
capacities = ["tight","moderate"]
instances_type = 0
number_of_instances = 0
exp_re = 30

for instance_size in 2:2, lat in latency, iso in isolation, c in capacities

    size = my_instance_sizes[instance_size]
    instances_type = instances_type + 1
    n = size["nodes"]
    d = size["densitiy"]
   if  iso == "strong"
        nfsh  = 0.1
        nosh = 0.1
    else
        nfsh  = 0.75
        nosh = 0.75
    end
        
    s  = size["slices"]
    k = size["commodities"]
    dp = size["DP_NFSs"]
    cp = size["CP_NFSs"]

    tries = 0
    i = 0
        
    while i <30
        i = i+1

        #creating instances
        number_of_instances = number_of_instances+1
        if number_of_instances >= 187
            instance = get_Instance(input_folder,number_of_instances)  
            warming_up =  "FEASIBLE"#warm_up(my_instance,"warm_tiny")


            if warming_up != "FEASIBLE" && warming_up != "OPTIMAL" && tries<=2000
                tries = tries+1
                i = i-1
                number_of_instances=number_of_instances-1
            elseif warming_up != "FEASIBLE" && warming_up != "OPTIMAL" && tries>2000
                i = 5000 
                number_of_instances=number_of_instances-1    
            elseif warming_up == "FEASIBLE" || warming_up == "OPTIMAL"
                tries = 0
                #solving instances
                for of in my_OFs, b in boosts, sol in my_solvers, vi in my_VIs,cut in my_cuts
                    my_parameters = Parameters()
                    my_parameters.class = "$(size["class"])_$(c)_$(lat)_$(iso)"            
                    my_parameters.instance_name = "instance_$(number_of_instances)_of_$(of)_boost_$(b)_solver_$(sol)_vi_$(vi)_cut_$(cut)_$(my_parameters.class)"              
                    my_parameters.valid_inequalities = vi
                    my_parameters.boost = b
                    #my_parameters.instance_class = ""            
                    my_parameters.solver = sol
                    my_parameters.instance_type = "instance_$(number_of_instances)_type_$(instances_type)_of_$(of)"
                    my_parameters.instance_number = number_of_instances
                    my_parameters.warming_up = false
                    my_parameters.number_of_phy_nodes = n
                    my_parameters.graph_density = d
                    my_parameters.link_capacity_ratio = c
                    my_parameters.node_capacity_ratio =c 
                    my_parameters.number_of_slice_requests =s
                    my_parameters.number_of_commodities_per_NS = k
                    my_parameters.number_of_CP_NFSs = cp 
                    my_parameters.number_of_DP_NFSs = dp
                    my_parameters.prob_sharing_NFS = nfsh
                    my_parameters.prob_sharing_node = nosh
                    my_parameters.objf = of                
                    my_parameters.cut_class = cut
                    my_parameters.cut_time = 0.0
                    my_parameters.number_of_cuts = 0
                    my_parameters.number_of_lazy_cuts= 0
                    my_parameters.lazy_time = 0.0            


            #creating model

                    #recreating new model cause jump starts from de LP already solved
                    my_parameters.relaxation=false
                    model = create_NSDP_complete_model(instance,my_parameters)                
                    # calling solver 
                    solve_NSDP_model(model,my_parameters,instance,"")
                   # get_MILP_LOG(result_folder,my_parameters,instance)
                    export_stat_instance(my_parameters,exp_re,"complete")
                    get_solution_mod(model,instance,result_folder,lat,number_of_instances,my_parameters)
                    model = nothing 

                    my_instance = solve_NSDP_heuristic(input_folder,number_of_instances, my_parameters.MILP_solution)
                    export_stat_instance_heu(my_parameters,exp_re,my_instance,my_parameters.MILP_solution)
                    get_solution_heu(my_instance,result_folder,lat,number_of_instances,my_parameters)

                end
            end
        end
    end
end    


In [1]:
my_instance.solution.time_found

UndefVarError: UndefVarError: my_instance not defined

In [46]:
##############################################################################
# Created by: Wesley DA SILVA COELHO
##############################################################################

##############################################################################
#                   INCLUDE PACKAGES
##############################################################################
using NBInclude #Package to include personal files",
using JuMP #Mathematical programming package : allows us to define the MILP",
using LightGraphs #Package to construct simple graphs",
using MetaGraphs #Package to construct complex graphs",
using GraphPlot

# path to some folders
input_folder = "../instances/heuristic/medium_small/"
include_folder = "../include/"
result_folder = "../instances/heuristic/medium_small/results/" 
@nbinclude(joinpath(include_folder,"NSDP_instance_generator.ipynb"))# code creator for 5GNSD instances
@nbinclude(joinpath(include_folder,"NSDP_structures.ipynb")) # our data structures 
@nbinclude(joinpath(include_folder,"NSDP_instance_reader.ipynb"))# file responsible for parseing new NSDP instances
@nbinclude(joinpath(include_folder,"NSDP_heuristic.ipynb")) # impementation of our model 


my_VIs = ["none"]
my_cuts = ["none"]
my_solvers = ["CPLEX"]
my_OFs = ["minNFS"]
boosts = [true]
my_parameters = Parameters()

my_instance_sizes = get_my_instance_sizes()
latency = ["low","high"]
isolation = ["strong","weak"]
capacities = ["tight","moderate"]
instances_type = 0
number_of_instances = 0
exp_re = 30
open_file_stat(exp_re)
open(joinpath(result_folder,"load_on_arcs.csv"), "w") do iooo
    open(joinpath(result_folder,"final_statistics.csv"), "w") do io
    open(joinpath(result_folder,"load_on_nodes.csv"), "w") do ioo    
        write(io,"instance;approach;class;graph_density;link_capacity_ratio;prob_sharing_NFS;")
        write(ioo,"instance;approach;node;node_type;class;graph_density;link_capacity_ratio;charge_node;prob_sharing_NFS\n")
        write(iooo,"instance;approach;arc;arc_type;class;graph_density;link_capacity_ratio;charge_arc;prob_sharing_NFS\n")
                write(io,"Objective_value;")   
                write(io,"number_NFSs;")
                write(io,"dis_NFS;")
                write(io,"cent_NFS;")


                write(io,"number_nodes;")
                write(io,"number_active_node;")
                write(io,"proportion_active_node;")

                write(io,"total_nodes_capacity;")
                write(io,"total_load_node;")
                write(io,"av_node_load);")
                write(io,"most_loaded_node;")


                write(io,"number_of_arcs;")
                write(io,"number_active_links;")
                write(io,"propotyion_number_active_links;")

                write(io,"total_arc_capacity;")
                write(io,"total_arc_load;")
                write(io,"av_link_load;")
                write(io,"most_loaded_arc;")
                write(io,"avr_lat\n")
        end
        end
        end
for instance_size in 4:4,te in 1:10
    iso = rand(isolation)
    lat = rand(latency)
    c = rand(capacities)
    size = my_instance_sizes[instance_size]
    instances_type = instances_type + 1
    n = size["nodes"]
    d = size["densitiy"]
   if  iso == "strong"
        nfsh  = 0.1
        nosh = 0.1
    else
        nfsh  = 0.75
        nosh = 0.75
    end
        
    s  = size["slices"]
    k = size["commodities"]
    dp = size["DP_NFSs"]
    cp = size["CP_NFSs"]

    tries = 0
    i = 0
        
    while i <1
        i = i+1

        #creating instances
        number_of_instances = number_of_instances+1
        if number_of_instances >= 1
            instance = get_Instance(input_folder,number_of_instances)  
            warming_up =  "FEASIBLE"#warm_up(my_instance,"warm_tiny")


            if warming_up != "FEASIBLE" && warming_up != "OPTIMAL" && tries<=2000
                tries = tries+1
                i = i-1
                number_of_instances=number_of_instances-1
            elseif warming_up != "FEASIBLE" && warming_up != "OPTIMAL" && tries>2000
                i = 5000 
                number_of_instances=number_of_instances-1    
            elseif warming_up == "FEASIBLE" || warming_up == "OPTIMAL"
                tries = 0
                #solving instances
                for of in my_OFs, b in boosts, sol in my_solvers, vi in my_VIs,cut in my_cuts
                    my_parameters = Parameters()
                    my_parameters.class = "$(size["class"])_$(c)_$(lat)_$(iso)"            
                    my_parameters.instance_name = "instance_$(number_of_instances)_of_$(of)_boost_$(b)_solver_$(sol)_vi_$(vi)_cut_$(cut)_$(my_parameters.class)"              
                    my_parameters.valid_inequalities = vi
                    my_parameters.boost = b
                    #my_parameters.instance_class = ""            
                    my_parameters.solver = sol
                    my_parameters.instance_type = "instance_$(number_of_instances)_type_$(instances_type)_of_$(of)"
                    my_parameters.instance_number = number_of_instances
                    my_parameters.warming_up = true
                    my_parameters.number_of_phy_nodes = n
                    my_parameters.graph_density = d
                    my_parameters.link_capacity_ratio = c
                    my_parameters.node_capacity_ratio =c 
                    my_parameters.number_of_slice_requests =s
                    my_parameters.number_of_commodities_per_NS = k
                    my_parameters.number_of_CP_NFSs = cp 
                    my_parameters.number_of_DP_NFSs = dp
                    my_parameters.prob_sharing_NFS = nfsh
                    my_parameters.prob_sharing_node = nosh
                    my_parameters.objf = of                
                    my_parameters.cut_class = cut
                    my_parameters.cut_time = 0.0
                    my_parameters.number_of_cuts = 0
                    my_parameters.number_of_lazy_cuts= 0
                    my_parameters.lazy_time = 0.0            


            #creating model

                    #recreating new model cause jump starts from de LP already solved
                    #my_parameters.relaxation=false
                    #model = create_NSDP_complete_model(instance,my_parameters)                
                    # calling solver 
                    #export_stat_instance(my_parameters,exp_re,"complete")
                   # get_solution_mod(model,instance,result_folder,lat,number_of_instances,my_parameters)
                    lb =  get_bounds(instance)
                    my_instance = solve_NSDP_heuristic(input_folder,number_of_instances, lb)
                    export_stat_instance_heu(my_parameters,exp_re,my_instance,lb)
                    get_solution_heu(my_instance,result_folder,lat,number_of_instances,my_parameters)

                end
            end
        end
    end
end    


InterruptException: InterruptException:

In [48]:
@nbinclude(joinpath(include_folder,"NSDP_heuristic.ipynb")) # impementation of our model 
instance = get_Instance(input_folder,number_of_instances)  
get_bounds(instance)

84.0

In [ ]:
my_parameters.warming_up = true
model = create_NSDP_complete_model(instance,my_parameters)                
                    # calling solver 
                    solve_NSDP_model(model,my_parameters,instance,"")

In [47]:
number_of_instances

1

In [38]:
@nbinclude(joinpath(include_folder,"NSDP_heuristic.ipynb")) # impementation of our model 
my_instance = solve_NSDP_heuristic(input_folder,number_of_instances)
my_instance.solution.installed_NFSs 

first solution 1066.0


Float16(77.0)

In [44]:
my_instance.solution.time_found 

Float16(1214.0)

In [39]:
@nbinclude(joinpath(include_folder,"NSDP_heuristic.ipynb")) # impementation of our model 
lb =  get_bounds(my_instance)

60.0

In [41]:
my_instance.solution.installed_NFSs 

Float16(77.0)